In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
from pipelines.data_preparation import CustomDataPreparationPipeline

In [5]:
# reading data
data = pd.read_csv("./data/train.csv")

# changing column names
new_columns = ["passenger_id", "home_planet", "cryo_sleep", "cabin", "destination", "age", "vip", "room_service", "food_court", "shopping_mall", "spa", "vr_deck", "name", "transported"]
data.columns = new_columns

# seperating X and y
X = data.drop(["transported"], axis=1)
y = data.transported

In [6]:
X.head()

,passenger_id,home_planet,cryo_sleep,cabin,destination,age,vip,room_service,food_court,shopping_mall,spa,vr_deck,name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [19]:
# splitting data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

<br>
<br>
<br>

In [20]:
data_preparation = CustomDataPreparationPipeline()

In [21]:
new_data = data_preparation.fit_transform(x_train, y_train)

In [22]:
new_data.head()

,home_planet,cryo_sleep,destination,age,vip,room_service,food_court,shopping_mall,spa,vr_deck,group,number_in_group,deck,num_in_cabin,side,total_spending,age_category,gender
1432,Mars,False,TRAPPIST-1e,31.0,False,1226.0,0.0,1.0,0.0,0.0,1510,1,F,291,S,1227.0,middle_adult,1.0
6858,Europa,False,TRAPPIST-1e,26.0,False,0.0,896.0,0.0,690.0,1.0,7253,1,D,225,P,1587.0,young_adult,1.0
4436,Earth,True,TRAPPIST-1e,24.0,False,0.0,0.0,0.0,0.0,0.0,4714,1,G,765,P,0.0,young_adult,1.0
7230,Earth,False,55 Cancri e,33.0,False,0.0,0.0,0.0,436.0,224.0,7727,1,E,507,S,660.0,middle_adult,0.0
2992,Mars,False,TRAPPIST-1e,21.0,False,1097.0,0.0,80.0,589.0,0.0,3237,1,D,104,P,1766.0,young_adult,1.0


In [23]:
new_data.isna().any().value_counts()

False    18
dtype: int64